<a href="https://colab.research.google.com/github/KonstantinBurkin/Vaccination_rates/blob/main/Vaccination_rates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>
<center><h1>Vaccination rates</h1></center>
<br>

<center><h3>Konstantin Burkin</h3></center>

<center> <i>21 December 2021</i> </center>

<br>  
<h2>Table of contents</h2> 

---
- <a href=#Introduction>Introduction</a> <br>
  - <a href=#Assignment>Assignment</a> <br>
  - <a href=#Brief-outline>Brief outline</a> <br>
- <a href=#Import-of-data-and-Python-libraries>Import of data and Python libraries</a> <br>
- <a href=#Exploratory-Data-Analysis>Exploratory Data Analysis</a> <br>
  - <a href=#Statistical-data-description>Statistical data description</a> <br>
  - <a href=#Filling-missing-values>Filling missing values</a> <br>
  - <a href=#Data-visualisation>Data visualisation</a> <br>
- <a href=#Prediction>Prediction</a> <br>
- <a href=#Results>Results</a> <br>

<br> 
<h2 id="Introduction">Introduction</h2>  

--- 

<div align="justify">This work was created and submitted as a final course project for MSU course "Data analysis with Python" in autumn semester 2021. The project was written in <a href="https://https://colab.research.google.com">Google Colab</a>, using Python <i>version 3.7.13</i>. This work is available in <a href="https://github.com/KonstantinBurkin/World_vaccination_data_analysis">my Github repository</a>, where it is possible to download Colab Notebook, check the code and reproduce my work.</div> <br>

<h3 id="Assignment">Assignment</h3> 

<div align="justify">The goal of this project is to find a dataframe, statistically describe it, remove or fill in the missing values, visualise the patterns in data and make a prediction on the basis of the obtained data. In this case, the dataframe describing COVID-19 World Vaccination Progress was chosen. After cleaning the data and visualising the rates of vaccinations in different countries and continents, the vaccinations data was fitted by non-linear regression and date was predicted when each country can achive herd immunity from COVID-19.</div> <br> 

<div align="justify">The result of this project is a dataframe containing id the names of the countries and the predicted date when the herd immunity is reached.

<br>  

<h3 id="Brief-outline">Brief outline</h3> 

<ul>
  <li>Setting up notebook environment: data import, data subsetting  import of libraries</li>
  <li>Exploration of dataframes: description of data types, description of column names, calculation of NAs.</li>
  <li>Filling missing values</li>
  <li>Visualisation of absolute vaccination rates across continents and fully vaccinated ratio for each country.</li>
  <li>Extrapolation and prediction of the date of fully vaccinated population.</li>

</ul> 

</div> 


<br> 
<h2 id="Import-of-data-and-Python-libraries">Import of data and Python libraries</h2>  

--- 

<div align="justify">The following data analysis includes several Python libraries for data analysis and ploting: </div>  

- Numpy
- Pandas
- Plotly
- Sklearn
- Scipy
- Datetime 
- Google colab   

<div align="justify">The dataframe was downloaded and read from the Github repository as a csv file. 

The dataframe is available on <a href="https://github.com/owid/covid-19-data">Github</a> or <a href="https://www.kaggle.com/gpreda/covid-world-vaccination-progress">Kaggle</a>. The original dataframe contains 104214 observations of 16 parameters. For this project only 8 parameters were subsetted and used.</div>

In [2]:
# Import libraries
from google.colab import output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sps
from datetime import timedelta, date, datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
import warnings
from google.colab import output
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

warnings.filterwarnings('ignore')

In [3]:
# Download data from Github
data_table = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv")

# Delete unnecessary columns
data_table.drop(labels=['daily_vaccinations_raw', 'daily_vaccinations', 'iso_code', 'daily_people_vaccinated', 'daily_people_vaccinated_per_hundred', \
                        'daily_vaccinations_per_million', 'total_boosters', 'total_boosters_per_hundred'], axis='columns', inplace=True)

# Modify the names of the columns
data_table.rename(columns={'location': 'country'}, inplace=True)

def column_name_preprocess(col_name):
    for char in ['total_', 'people_']:
        col_name = col_name.replace(char, '')
        col_name = col_name.replace('_per_hundred', '_ratio')
    return col_name

column_names = data_table.columns
data_table.columns = [column_name_preprocess(col_name).title() for col_name in column_names]

# List of all countries and continents
countries = list(data_table.Country.unique())     

In [ ]:
# # Display the 5 random rows from dataframe
# data_table.sample(5)

<center><p>The 5 random rows of the dataframe</p> 

<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-f6o3{font-family:Nunito, sans-serif !important;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-bd7x{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;text-align:center;vertical-align:top}
.tg .tg-f1yk{font-family:Nunito, sans-serif !important;text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-f6o3"><span style="font-weight:bold">Country</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Date</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinations</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinated</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Fully_Vaccinated</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinations_Ratio</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinated_Ratio</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Fully_Vaccinated_Ratio</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-bd7x">Canada</td>
    <td class="tg-bd7x">2021-07-22</td>
    <td class="tg-bd7x">47056217.0</td>
    <td class="tg-bd7x">26783674.0</td>
    <td class="tg-bd7x">20268756.0</td>
    <td class="tg-bd7x">123.61</td>
    <td class="tg-bd7x">70.36</td>
    <td class="tg-bd7x">53.24</td>
  </tr>
  <tr>
    <td class="tg-f1yk">Italy</td>
    <td class="tg-f1yk">2021-05-31</td>
    <td class="tg-f1yk">35434142.0</td>
    <td class="tg-f1yk">23815455.0</td>
    <td class="tg-f1yk">12280305.0</td>
    <td class="tg-f1yk">58.70</td>
    <td class="tg-f1yk">39.45</td>
    <td class="tg-f1yk">20.34</td>
  </tr>
  <tr>
    <td class="tg-bd7x">Palestine</td>
    <td class="tg-bd7x">2022-02-15</td>
    <td class="tg-bd7x">NaN</td>
    <td class="tg-bd7x">NaN</td>
    <td class="tg-bd7x">NaN</td>
    <td class="tg-bd7x">NaN</td>
    <td class="tg-bd7x">NaN</td>
    <td class="tg-bd7x">NaN</td>
  </tr>
  <tr>
    <td class="tg-f1yk">Bangladesh</td>
    <td class="tg-f1yk">2021-12-29</td>
    <td class="tg-f1yk">NaN</td>
    <td class="tg-f1yk">NaN</td>
    <td class="tg-f1yk">NaN</td>
    <td class="tg-f1yk">NaN</td>
    <td class="tg-f1yk">NaN</td>
    <td class="tg-f1yk">NaN</td>
  </tr>
  <tr>
    <td class="tg-bd7x">Montenegro</td>
    <td class="tg-bd7x">2022-03-28</td>
    <td class="tg-bd7x">668025.0</td>
    <td class="tg-bd7x">289643.0</td>
    <td class="tg-bd7x">281511.0</td>
    <td class="tg-bd7x">106.36</td>
    <td class="tg-bd7x">46.12</td>
    <td class="tg-bd7x">44.82</td>
  </tr>
</tbody>
</table></center>

<br> 
<h2 id="Exploratory-Data-Analysis">Exploratory Data Analysis</h2>  

--- 

<h3 id="Statistical-data-description">Statistical data description</h3>  

<div align="justify">The dataframe describes vaccination rates in 235 countries across the world. The vaccinations began in the end of 2020. Vaccination programs still exist at spring of 2022. 
There are several parameters that describe vaccination rates. The names of the columns of the data frame that descibe each type of parameter are listed below with the description. It is important to underline that the number of vaccinations could be larger than the population size since many vaccines require two shots. Moreover, many people traveled aboad to recieve a better or additional vaccine. </div>  

<center><p>Table of variables</p> 

<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-dgtn{font-family:Nunito, sans-serif !important;font-size:100%;font-style:italic;font-weight:bold;
  text-align:center;vertical-align:top}
.tg .tg-7m2y{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;text-align:center;
  vertical-align:top}
.tg .tg-uq8q{font-family:Nunito, sans-serif !important;font-size:100%;font-weight:bold;text-align:center;
  vertical-align:top}
.tg .tg-mqzu{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;text-align:center;
  vertical-align:middle}
.tg .tg-r28n{font-family:Nunito, sans-serif !important;font-size:100%;text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-uq8q"><span style="font-weight:bold">Variable</span></th>
    <th class="tg-uq8q"><span style="font-weight:bold">Description</span></th>
    <th class="tg-uq8q"><span style="font-weight:bold">Datatype</span></th>
    <th class="tg-uq8q"><span style="font-weight:bold">NAs</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-7m2y">Country</td>
    <td class="tg-mqzu"><span style="font-weight:400;font-style:normal">The country for which the vaccination rate is provided</span></td>
    <td class="tg-7m2y">object</td>
    <td class="tg-7m2y">0</td>
  </tr>
  <tr>
    <td class="tg-r28n">Date</td>
    <td class="tg-r28n">Date for the data entry</td>
    <td class="tg-r28n">object</td>
    <td class="tg-r28n">0</td>
  </tr>
  <tr>
    <td class="tg-7m2y">Vaccinations</td>
    <td class="tg-7m2y">The absolute number of immunizations in the country</td>
    <td class="tg-7m2y">float64</td>
    <td class="tg-7m2y">50241</td>
  </tr>
  <tr>
    <td class="tg-r28n">Vaccinated</td>
    <td class="tg-r28n">Total number of people vaccinated. A person, depending on the immunization scheme, will receive one or more (typically 2) vaccines; at a certain moment, the number of vaccination might be larger than the number of people</td>
    <td class="tg-r28n">float64</td>
    <td class="tg-r28n">52683</td>
  </tr>
  <tr>
    <td class="tg-7m2y">Fully_Vaccinated</td>
    <td class="tg-7m2y">The number of people that received the entire set of immunization according to the immunization scheme (typically 2)</td>
    <td class="tg-7m2y">float64</td>
    <td class="tg-7m2y">55236</td>
  </tr>
  <tr>
    <td class="tg-r28n">Vaccinations_Ratio</td>
    <td class="tg-r28n">The ratio between vaccination number and total population up to the date in the country</td>
    <td class="tg-r28n">float64</td>
    <td class="tg-r28n">50241</td>
  </tr>
  <tr>
    <td class="tg-7m2y">Vaccinated_Ratio</td>
    <td class="tg-7m2y">The ratio between population immunized and total population up to the date in the country</td>
    <td class="tg-7m2y">float64</td>
    <td class="tg-7m2y">52683</td>
  </tr>
  <tr>
    <td class="tg-r28n">Fully_Vaccinated_Ratio</td>
    <td class="tg-r28n">The ratio between population fully immunized and total population up to the date in the country</td>
    <td class="tg-r28n">float64</td>
    <td class="tg-r28n">55236</td>
  </tr>
</tbody>
</table>

</center>

In [ ]:
# # number of NAs in each column
# data_table.isna().sum()

In [ ]:
# # statistical characteristics of the dataset
# data_table.describe().loc[['mean','min', 'max', '25%', '50%', '75%']].round(2)

<center><p>Statistical description of numeric data</p> 
<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-ehj1{font-family:Nunito, sans-serif !important;font-weight:bold;text-align:center;vertical-align:middle}
.tg .tg-42x8{font-family:Nunito, sans-serif !important;text-align:center;vertical-align:middle}
.tg .tg-f6o3{font-family:Nunito, sans-serif !important;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-a2mc{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-weight:bold;text-align:center;
  vertical-align:middle}
.tg .tg-nzir{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;text-align:center;vertical-align:middle
  }
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-f6o3"></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinations</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinated</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Fully_Vaccinated</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinations_Ratio</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Vaccinated_Ratio</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Fully_Vaccinated_Ratio</span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Fully_Vaccinated_Ratio</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-a2mc">mean</td>
    <td class="tg-nzir">2.12e+08</td>
    <td class="tg-nzir">1.03e+08</td>
    <td class="tg-nzir">8.57e+07</td>
    <td class="tg-nzir">86.25</td>
    <td class="tg-nzir">42.69</td>
    <td class="tg-nzir">37.19</td>
    <td class="tg-nzir">37.19</td>
  </tr>
  <tr>
    <td class="tg-ehj1">min</td>
    <td class="tg-42x8">0.00e+00</td>
    <td class="tg-42x8">0.00e+00</td>
    <td class="tg-42x8">1.00e+00</td>
    <td class="tg-42x8">0.00</td>
    <td class="tg-42x8">0.00</td>
    <td class="tg-42x8">0.00</td>
    <td class="tg-42x8">0.00</td>
  </tr>
  <tr>
    <td class="tg-a2mc">max</td>
    <td class="tg-nzir">1.17e+10</td>
    <td class="tg-nzir">5.17e+09</td>
    <td class="tg-nzir">4.71e+09</td>
    <td class="tg-nzir">355.75</td>
    <td class="tg-nzir">124.88</td>
    <td class="tg-nzir">122.94</td>
    <td class="tg-nzir">122.94</td>
  </tr>
  <tr>
    <td class="tg-ehj1">25%</td>
    <td class="tg-42x8">8.28e+05</td>
    <td class="tg-42x8">4.88e+05</td>
    <td class="tg-42x8">3.87e+05</td>
    <td class="tg-42x8">17.83</td>
    <td class="tg-42x8">12.25</td>
    <td class="tg-42x8">7.49</td>
    <td class="tg-42x8">7.49</td>
  </tr>
  <tr>
    <td class="tg-a2mc">50%</td>
    <td class="tg-nzir">6.36e+06</td>
    <td class="tg-nzir">3.77e+06</td>
    <td class="tg-nzir">2.95e+06</td>
    <td class="tg-nzir">75.80</td>
    <td class="tg-nzir">45.47</td>
    <td class="tg-nzir">35.78</td>
    <td class="tg-nzir">35.78</td>
  </tr>
  <tr>
    <td class="tg-ehj1">75%</td>
    <td class="tg-42x8">3.96e+07</td>
    <td class="tg-42x8">2.21e+07</td>
    <td class="tg-42x8">1.82e+07</td>
    <td class="tg-42x8">142.02</td>
    <td class="tg-42x8">69.76</td>
    <td class="tg-42x8">63.88</td>
    <td class="tg-42x8">63.88</td>
  </tr>
</tbody>
</table></center>

<br>
<div align="justify">Three columns that describe ratio of vaccinated people (Vaccinations_Ratio,	Vaccinated_Ratio,	and Fully_Vaccinated_Ratio) contain values of more than 100%, since some vaccinations require two immunization shots. However, the fact that in some countries there could be more fully vaccinated people that the amount of population (Fully_Vaccinated_Ratio > 100 %) seems erroneous. </div> 




<h3 id="Filling-missing-values">Filling missing values</h3>  

<div align="justify">It is unlikely that vaccination rates can drasticaly change between two points of data entry. Therefore, it seems logical to fill missing values with linear interpolation. To be certain that filling of missing values was correct and linear approximation was pertinent, it is possible to see the visualisation of the plots in the next section.</div>  

In [4]:
# Filling missing values with data using linear approximation

for country in countries:
    data_table.loc[data_table['Country'] == country] = \
    data_table.loc[data_table['Country'] == country].interpolate(method='linear')

<h3 id="Data-visualisation">Data visualisation</h3> 

<div align="justify">The first graph presents absolute vaccinations rates across six continents. Filling missing values did not result in any kind of unusual values or outliers. Therefore, it is possible to build predictions for the end of vaccinations.</div>  
<br>
<div align="justify">Before going to the next step it is interesting to notice that Europe, Asia, and North America were the first countries to develop vaccines against COVID-19 and deploy a full-scale vaccination program. Other continents, like Africa, South America, and Australia lagged behind in vaccination rates, since they did not have neither developed pharm industry nor enough resources, and they had to wait for vaccine supplies from developed countries. These patterns can be seen in the graphs. Europe, Asia, and North America have high numbers of vaccination rates since the end of 2020.</div>  


In [12]:
# make graph with continental rates
figure = make_subplots(
                       rows=2, 
                       cols=3, 
                       subplot_titles=("Africa", "Asia", "Europe", "Australia", "South America", "North America"),
                       horizontal_spacing = 0.07, 
                       )

# Africa
figure.add_trace(go.Scatter(
                            x=list(data_table[data_table['Country'] == "Africa"].Date), 
                            y=data_table[data_table['Country'] == "Africa"].Vaccinations
                            ), 
                 row=1, 
                 col=1
                 )
figure.update_xaxes(title_text='Date', row=1, col=1)
figure.update_yaxes(title_text='Vaccitantions', row=1, col=1)

# Asia
figure.add_trace(go.Scatter(
                            x=list(data_table[data_table['Country'] == "Asia"].Date), 
                            y=data_table[data_table['Country'] == "Asia"].Vaccinations
                            ), 
                 row=1, 
                 col=2
                 )
figure.update_xaxes(title_text='Date', row=1, col=2)
figure.update_yaxes(title_text='Vaccitantions', row=1, col=2)

# Europe
figure.add_trace(go.Scatter(
                            x=list(data_table[data_table['Country'] == "Europe"].Date), 
                            y=data_table[data_table['Country'] == "Europe"].Vaccinations
                            ), 
                 row=1, 
                 col=3
                 )
figure.update_xaxes(title_text='Date', row=1, col=3)
figure.update_yaxes(title_text='Vaccitantions', row=1, col=3)

# Australia
figure.add_trace(go.Scatter(
                            x=list(data_table[data_table['Country'] == "Australia"].Date), 
                            y=data_table[data_table['Country'] == "Australia"].Vaccinations
                            ), 
                 row=2, 
                 col=1
                 )
figure.update_xaxes(title_text='Date', row=2, col=1)
figure.update_yaxes(title_text='Vaccitantions', row=2, col=1)

# South America
figure.add_trace(go.Scatter(
                            x=list(data_table[data_table['Country'] == "South America"].Date), 
                            y=data_table[data_table['Country'] == "South America"].Vaccinations
                            ), 
                 row=2, 
                 col=2
                 )
figure.update_xaxes(title_text='Date', row=2, col=2)
figure.update_yaxes(title_text='Vaccitantions', row=2, col=2)

# North America
figure.add_trace(go.Scatter(
                            x=list(data_table[data_table['Country'] == "North America"].Date), 
                            y=data_table[data_table['Country'] == "North America"].Vaccinations
                            ), 
                 row=2, 
                 col=3
                 )
figure.update_xaxes(title_text='Date', row=2, col=3)
figure.update_yaxes(title_text='Vaccitantions', row=2, col=3)

figure.update_layout(
                     font_family="'Nunito', sans-serif",
                     title={'text':"Correlations between sales and other columns", 'y':0.99, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
                     showlegend=False, 
                     margin=dict(l=40, r=10, t=60, b=60)
                     )

figure.show()

In [6]:
# Make a list of graphs for each country

z = []
for number in range(len(countries)):
    fig1 = go.Figure()
    
    fig1.add_trace(go.Scatter(
                              x=data_table[data_table['Country'] == countries[number]].Date,
                              y=data_table[data_table['Country'] == countries[number]].Fully_Vaccinated_Ratio
                              )
                   )
    fig1.update_layout(
                       font_family="'Nunito', sans-serif",
                       title={'text': f"Vaccination rates in {countries[number]}", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                       xaxis_title='Date',
                       yaxis_title='Vaccinations',
                       showlegend=False
                       )
    z.append(fig1)

<div align="justify">The ratio of fully vaccinated people is ploted below for each country. It is possible to choose the country of interest using the dropdown menu. Again, it is evident that despite the fact that vaccinations began in the end of 2020 many countries started vaccinations programs much later. At that time only developed asian, european and north american countries could financially and logisticaly afford to begin vaccinations. Less developed countries started to receive vaccines later in the beginning of 2021.</div>  


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                         x=data_table[data_table['Country'] == countries[0]].Date,
                         y=data_table[data_table['Country'] == countries[0]].Fully_Vaccinated_Ratio
                         )
              )
fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  title={'text': "Ratio of fully vaccinated people", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Date',
                  yaxis_title='Population ratio, %',
                  showlegend=False
                  )

fig.update_layout(
                  height = 500,
                  margin=dict(l=40, r=20, t=60, b=20),
                  updatemenus=[
                               dict(
                                   buttons=list([dict(
                                                       args=[{'x': [dat.x for dat in z[number].data],'y': [dat.y for dat in z[number].data]}],
                                                       label=f"{countries[number]}",
                                                       method="restyle") for number in range(len(countries))
                                                  ]
                                                 ),
                
                                    direction="down",
                                    showactive=True,
                                    yanchor='middle',
                                    y=1.08,
                                    xanchor='center',
                                    x=0.85,
                                    )
                               ]
                  )


fig.show()

<br> 
<h2 id="Prediction">Prediction</h2>  

--- 

<div align="justify">To predict the end of vaccinations non-linear regression was implemented to fit the vaccination rates in each country. The logistic function was used:</div>  



$$ f(x) = \frac L {1+{e}^{-k * (x-x_0)}} \ $$  

<div align="justify">This type of non-linear regression fitted curves for 199 countries with values of \(R^2 > 0.9\). The example of vaccination rates in Ireland proves that this function perfectly describes the data. The data from countries that could not be fitted with logistic regression were not analyzed in the following work. Generally, these countries did not have enough vaccinations data.  </div> 

In [ ]:
# define sigmoidal curve
def func(x, L, k, x0):
    return L / (1 + np.exp(-k * (x-x0) ))

# define reverse sigmoidal curve
def func_predict(y, L, k, x0):
    return -np.log(L/y-1)/k + x0

predictions = []
r2 = []

# iteratively fit logistic function to each country;
# calculate predictions and r2-value
for country in countries:
    subset = data_table[data_table['Country'] == country].dropna()
    x = list(range(subset.shape[0]))
    y = subset.Fully_Vaccinated_Ratio
    if country == "Tokelau":
        r2.append(0)
        predictions.append(0)
        continue
    try:
        popt, pcov = curve_fit(func, x, y)
    except RuntimeError:
        r2.append(0)
        predictions.append(0)
        continue
    r2.append(r2_score(y, func(x, *popt).round(3)))
    try:
        date = datetime.strptime((np.array(subset.Date)[0]), '%Y-%m-%d') + timedelta(days=func_predict(75, *popt).round(0))
        predictions.append( date.strftime("%d %b %Y"))
    except ValueError:
        predictions.append(0)

r2 = np.round(np.array(r2),3)
countries = pd.DataFrame(data=countries, columns=['Country'])
r2 = pd.DataFrame(data=r2, columns=['R2'])
predictions = pd.DataFrame(data=predictions, columns=['Prediction'])

# concatenate countries, r2, and predictions into one dataframe;
# subset dataframe to get values with r2 > 0.9
result = pd.concat([countries, r2, predictions], axis=1)
result = result[result.R2 > 0.1].dropna()[result.Prediction != 0]

In [ ]:
# plot example of ireland and fitted curve
subset = data_table[data_table['Country'] == 'Ireland'].dropna()
x = list(range(subset.shape[0]))
y = subset.Fully_Vaccinated_Ratio

popt, pcov = curve_fit(func, x, y)


fig = go.Figure()
fig.add_trace(go.Scatter(
                         x=data_table[data_table['Country'] == 'Ireland'].Date,
                         y=data_table[data_table['Country'] == 'Ireland'].Fully_Vaccinated_Ratio,
                         name="Real data"
                         )
              )

fig.add_trace(go.Scatter(
                         x=subset.Date,
                         y=func(x, *popt),
                         name="Regression"
                         )
              )

fig.update_layout(
                  font_family="'Nunito', sans-serif",
                  title={'text': f"Vaccination rates in Ireland", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  height=400,
                  margin=dict(l=40, r=20, t=50, b=60), 
                  xaxis_title='Date',
                  yaxis_title='Vaccinations',
                  showlegend=True,
                  legend={}
                  )

# add anotation with r2
fig.add_annotation(dict(
                        x=1.1,
                        y=1.1,
                        showarrow=False,
                        text="$R^2 = 0.999$",
                        textangle=0,
                        xanchor='right',
                        xref="paper",
                        yref="paper"))

fig.show()

<div align="justify"> After the data was fitted with non-linear regression, it is interesting to get information about the date, when vaccination programs can be closed. To do that, it is necessary to determine the threshold, after which, the vaccinations can be stopped. The threshold, in a given population, is the point where the disease reaches a steady state, which means that the infection level is neither growing nor declining exponentially. The threshold is evaluated using the formula below.</div>

$$ p_c = 1 - \frac 1 R_0\ $$

 
<div align="justify">where \(R_0\) \(-\) the basic reproduction number, the average number of new infections caused by each case in a population where each individual is equally likely to come into contact with any other susceptible individual in the population; <br> \(P_c\) \(-\) the critical proportion of the population needed to be immune to stop the transmission of disease, which is the same as the "herd immunity threshold" (HIT).
Information about herd immunity thresholds was found <a href="https://en.wikipedia.org/wiki/Herd_immunity#Theoretical_basis">here</a>.</div>

<center><p>Values of \(R_0\) and herd immunity thresholds (HITs) of well-known infectious diseases prior to intervention</p> 

<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-9myl{font-family:Nunito, sans-serif !important;font-size:100%;text-align:left;text-decoration:underline;
  vertical-align:top}
.tg .tg-dgtn{font-family:Nunito, sans-serif !important;font-size:100%;font-style:italic;font-weight:bold;
  text-align:center;vertical-align:top}
.tg .tg-ss8n{font-family:Nunito, sans-serif !important;font-size:100%;text-align:left;vertical-align:top}
.tg .tg-7m2y{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;text-align:center;
  vertical-align:top}
.tg .tg-afns{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;text-align:left;
  vertical-align:middle}
.tg .tg-uq8q{font-family:Nunito, sans-serif !important;font-size:100%;font-weight:bold;text-align:center;
  vertical-align:top}
.tg .tg-09c9{font-family:Nunito, sans-serif !important;font-size:100%;text-align:left;vertical-align:middle}
.tg .tg-ifcf{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;font-style:italic;
  text-align:left;text-decoration:underline;vertical-align:top}
.tg .tg-9bu0{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;text-align:left;
  vertical-align:top}
.tg .tg-mqzu{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;text-align:center;
  vertical-align:middle}
.tg .tg-kpux{font-family:Nunito, sans-serif !important;font-size:100%;text-align:center;vertical-align:middle}
.tg .tg-f8ma{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-size:100%;text-align:left;
  text-decoration:underline;vertical-align:top}
.tg .tg-r28n{font-family:Nunito, sans-serif !important;font-size:100%;text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-uq8q">Disease</th>
    <th class="tg-uq8q"><span style="font-weight:bold">Transmission</span></th>
    <th class="tg-uq8q">\(R_0\)</th>
    <th class="tg-uq8q">HIT, %</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Measles" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Measles</span></a></td>
    <td class="tg-9bu0">Aerosol</td>
    <td class="tg-mqzu">12-18</td>
    <td class="tg-mqzu">92-94</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Chickenpox" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Chickenpox</span></a> (varicella)</td>
    <td class="tg-09c9">Aerosol</td>
    <td class="tg-kpux">10-12</td>
    <td class="tg-kpux">90-92</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Mumps" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Mumps</span></a></td>
    <td class="tg-9bu0">Respiratory droplets</td>
    <td class="tg-mqzu">10-12</td>
    <td class="tg-mqzu">90-92</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/COVID-19" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">COVID-19</span></a> <span style="text-decoration:underline">(</span><a href="https://en.wikipedia.org/wiki/Investigations_into_the_origin_of_COVID-19" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">ancestral strain</span></a><span style="text-decoration:underline">)</span></td>
    <td class="tg-ss8n">Respiratory droplets and aerosol</td>
    <td class="tg-r28n">2.4-3.4</td>
    <td class="tg-r28n">58-71</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/COVID-19" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">COVID-19</span></a> <span style="text-decoration:underline">(</span><a href="https://en.wikipedia.org/wiki/SARS-CoV-2_Alpha_variant" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Alpha variant</span></a><span style="text-decoration:underline">)</span></td>
    <td class="tg-9bu0">Respiratory droplets and aerosol</td>
    <td class="tg-7m2y">4-5</td>
    <td class="tg-7m2y">75-80</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/COVID-19" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">COVID-19</span></a> <span style="text-decoration:underline">(</span><a href="https://en.wikipedia.org/wiki/SARS-CoV-2_Delta_variant" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Delta variant</span></a><span style="text-decoration:underline">)</span></td>
    <td class="tg-ss8n">Respiratory droplets and aerosol</td>
    <td class="tg-r28n">5.1</td>
    <td class="tg-r28n">80</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/COVID-19" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">COVID-19</span></a> (<a href="https://en.wikipedia.org/wiki/SARS-CoV-2_Omicron_variant" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Omicron variant</span></a>)</td>
    <td class="tg-afns">Respiratory droplets and aerosol</td>
    <td class="tg-mqzu">9.5</td>
    <td class="tg-mqzu">89</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Rubella" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Rubella</span></a></td>
    <td class="tg-09c9">Respiratory droplets</td>
    <td class="tg-kpux">6-7</td>
    <td class="tg-kpux">83-86</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Polio" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Polio</span></a></td>
    <td class="tg-9bu0">Fecal-oral route</td>
    <td class="tg-mqzu">5-7</td>
    <td class="tg-mqzu">80-86</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Whooping_cough" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Pertussis</span></a></td>
    <td class="tg-09c9">Respiratory droplets</td>
    <td class="tg-kpux">5.5</td>
    <td class="tg-kpux">82</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Smallpox" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Smallpox</span></a></td>
    <td class="tg-afns">Respiratory droplets</td>
    <td class="tg-mqzu">3.5-6.0</td>
    <td class="tg-mqzu">71-83</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/HIV/AIDS" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">HIV/AIDS</span></a></td>
    <td class="tg-ss8n">Body fluids</td>
    <td class="tg-kpux">2-5</td>
    <td class="tg-kpux">50-80</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Severe_acute_respiratory_syndrome" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">SARS</span></a></td>
    <td class="tg-afns">Respiratory droplets</td>
    <td class="tg-mqzu">2-4</td>
    <td class="tg-mqzu">50-75</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Diphtheria" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Diphtheria</span></a></td>
    <td class="tg-ss8n">Saliva</td>
    <td class="tg-r28n">1.7-4.3</td>
    <td class="tg-r28n">41-77</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Common_cold" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Common cold</span></a></td>
    <td class="tg-afns">Respiratory droplets</td>
    <td class="tg-mqzu">2-3</td>
    <td class="tg-mqzu">50-67</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Monkeypox" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Monkeypox</span></a></td>
    <td class="tg-09c9">Physical contact, body fluids, respiratory droplets</td>
    <td class="tg-r28n">1.5-2.7</td>
    <td class="tg-r28n">31-63</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Influenza" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Influenza</span></a> (<a href="https://en.wikipedia.org/wiki/Spanish_flu" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">1918 pandemic strain</span></a>)</td>
    <td class="tg-afns">Respiratory droplets</td>
    <td class="tg-mqzu">2</td>
    <td class="tg-mqzu">50</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Ebola" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Ebola</span></a> (<a href="https://en.wikipedia.org/wiki/2014_Ebola_outbreak" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">2014 outbreak</span></a>)</td>
    <td class="tg-09c9">Body fluids</td>
    <td class="tg-r28n">1.4-1.8</td>
    <td class="tg-r28n">31-44</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Influenza" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Influenza</span></a> (<a href="https://en.wikipedia.org/wiki/2009_swine_flu_pandemic" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">2009 pandemic</span></a> <a href="https://en.wikipedia.org/wiki/Pandemic_H1N1/09_virus" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">strain</span></a>)</td>
    <td class="tg-afns">Respiratory droplets</td>
    <td class="tg-7m2y">1.3-2.0</td>
    <td class="tg-7m2y">25-51</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Influenza" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Influenza</span></a> (seasonal strains)</td>
    <td class="tg-09c9">Respiratory droplets</td>
    <td class="tg-r28n">1.2-1.4</td>
    <td class="tg-r28n">17-29</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/Andes_orthohantavirus" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Andes hantavirus</span></a></td>
    <td class="tg-afns">Respiratory droplets and body fluids</td>
    <td class="tg-7m2y">0.8-1.6</td>
    <td class="tg-7m2y">0-36</td>
  </tr>
  <tr>
    <td class="tg-9myl"><a href="https://en.wikipedia.org/wiki/Nipah_virus" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">Nipah virus</span></a></td>
    <td class="tg-09c9">Body fluids</td>
    <td class="tg-kpux">0.5</td>
    <td class="tg-kpux">0</td>
  </tr>
  <tr>
    <td class="tg-f8ma"><a href="https://en.wikipedia.org/wiki/MERS" target="_blank" rel="noopener noreferrer"><span style="text-decoration:none">MERS</span></a></td>
    <td class="tg-afns">Respiratory droplets</td>
    <td class="tg-7m2y">0.3-0.8</td>
    <td class="tg-mqzu">0</td>
  </tr>
</tbody>
</table>
</center>

<br>
<div align="justify">For this project, the lowest threshold of 75% for the alpha variant of COVID-19 was calculated. Using logistic regression, the dates when immunizations reach the threshold were found (shown below). It is important to underline that out of 199 curves (\(R^2 > 0.9\)) that described vaccination rates in 199 countries only 38 countries reached or will reach in nearest future the threshold of herd immunity. The logistic regression for data from other countries shows that herd immunity cannot be achieved according to the current trend in vaccination rates. </div>
<div align="justify">However, it is important to point out that a lot of people could get immunity from COVID-19 after having had the disease. That could have lead to lower percentage of the population needed for reaching herd immunity threshold. Unfortunately, in this project this factor was not yet taken into account.</div>
 

In [ ]:
# # the code is represented in the table below
# result

<center><p>Predicted dates of the end of vaccinations</p> 
<style type="text/css">
.tg  {border:none;border-collapse:collapse;border-color:#ccc;border-spacing:0;}
.tg td{background-color:#fff;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{background-color:#f0f0f0;border-color:#ccc;border-style:solid;border-width:0px;color:#333;
  font-family:Nunito, sans-serif;font-size:14px;font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-ehj1{font-family:Nunito, sans-serif !important;font-weight:bold;text-align:center;vertical-align:middle}
.tg .tg-42x8{font-family:Nunito, sans-serif !important;text-align:center;vertical-align:middle}
.tg .tg-f6o3{font-family:Nunito, sans-serif !important;font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-m39u{font-family:Nunito, sans-serif !important;font-style:italic;font-weight:bold;text-align:center;
  vertical-align:top}
.tg .tg-quxf{background-color:#ffffff;text-align:center;vertical-align:top}
.tg .tg-5oz6{font-family:Nunito, sans-serif !important;font-style:italic;font-weight:bold;text-align:center;
  vertical-align:middle}
.tg .tg-i81m{background-color:#ffffff;text-align:center;vertical-align:top}
.tg .tg-epaa{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;font-style:italic;text-align:center;
  vertical-align:middle}
.tg .tg-nzir{background-color:#f9f9f9;font-family:Nunito, sans-serif !important;text-align:center;vertical-align:middle
  }
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-f6o3">Country</th>
    <th class="tg-f6o3"><span style="font-weight:700;font-style:italic"><b>\(R^2\)</b></span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Prediction</span></th>
    <th class="tg-i81m"></th>
    <th class="tg-f6o3">Country</th>
    <th class="tg-f6o3"><span style="font-weight:700;font-style:italic"><b>\(R^2\)</b></span></th>
    <th class="tg-f6o3"><span style="font-weight:bold">Prediction</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-nzir">Argentina</td>
    <td class="tg-nzir">0.998</td>
    <td class="tg-nzir">01 Jan 2022</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Isle of Man</td>
    <td class="tg-nzir">0.986</td>
    <td class="tg-nzir">03 Oct 2021</td>
  </tr>
  <tr>
    <td class="tg-42x8">Australia</td>
    <td class="tg-42x8">0.999</td>
    <td class="tg-42x8">12 Dec 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">Italy</td>
    <td class="tg-42x8">0.997</td>
    <td class="tg-42x8">23 Nov 2021</td>
  </tr>
  <tr>
    <td class="tg-nzir">Bangladesh</td>
    <td class="tg-nzir">0.994</td>
    <td class="tg-nzir">23 May 2022</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Japan</td>
    <td class="tg-nzir">0.998</td>
    <td class="tg-nzir">08 Nov 2021</td>
  </tr>
  <tr>
    <td class="tg-42x8">Belgium</td>
    <td class="tg-42x8">0.998</td>
    <td class="tg-42x8">12 Oct 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">Kuwait</td>
    <td class="tg-42x8">0.997</td>
    <td class="tg-42x8">30 Jan 2022</td>
  </tr>
  <tr>
    <td class="tg-nzir">Brazil</td>
    <td class="tg-nzir">0.998</td>
    <td class="tg-nzir">31 Mar 2022</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Luxembourg</td>
    <td class="tg-nzir">0.991</td>
    <td class="tg-nzir">25 Sep 2022</td>
  </tr>
  <tr>
    <td class="tg-42x8">Brunei</td>
    <td class="tg-42x8">0.997</td>
    <td class="tg-42x8">21 Nov 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">Macao</td>
    <td class="tg-42x8">0.986</td>
    <td class="tg-42x8">01 Feb 2022</td>
  </tr>
  <tr>
    <td class="tg-nzir">Cambodia</td>
    <td class="tg-nzir">0.998</td>
    <td class="tg-nzir">03 Nov 2021</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Malaysia</td>
    <td class="tg-nzir">0.999</td>
    <td class="tg-nzir">03 Nov 2021</td>
  </tr>
  <tr>
    <td class="tg-42x8">Canada</td>
    <td class="tg-42x8">0.993</td>
    <td class="tg-42x8">19 Sep 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">Malta</td>
    <td class="tg-42x8">0.994</td>
    <td class="tg-42x8">03 Aug 2021</td>
  </tr>
  <tr>
    <td class="tg-nzir">Chile</td>
    <td class="tg-nzir">0.983</td>
    <td class="tg-nzir">07 Sep 2021</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Nepal</td>
    <td class="tg-nzir">0.986</td>
    <td class="tg-nzir">21 Jul 2022</td>
  </tr>
  <tr>
    <td class="tg-42x8">Congo</td>
    <td class="tg-42x8">0.984</td>
    <td class="tg-42x8">12 Apr 2023</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">New Zealand</td>
    <td class="tg-42x8">0.997</td>
    <td class="tg-42x8">31 Dec 2021</td>
  </tr>
  <tr>
    <td class="tg-nzir">Costa Rica</td>
    <td class="tg-nzir">0.993</td>
    <td class="tg-nzir">17 Feb 2022</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Peru</td>
    <td class="tg-nzir">0.998</td>
    <td class="tg-nzir">23 Feb 2022</td>
  </tr>
  <tr>
    <td class="tg-42x8">Cuba</td>
    <td class="tg-42x8">0.993</td>
    <td class="tg-42x8">22 Nov 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">Portugal</td>
    <td class="tg-42x8">0.999</td>
    <td class="tg-42x8">06 Sep 2021</td>
  </tr>
  <tr>
    <td class="tg-nzir">Denmark</td>
    <td class="tg-nzir">0.996</td>
    <td class="tg-nzir">01 Oct 2021</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Samoa</td>
    <td class="tg-nzir">0.962</td>
    <td class="tg-nzir">24 May 2022</td>
  </tr>
  <tr>
    <td class="tg-42x8">Faeroe Islands</td>
    <td class="tg-42x8">0.996</td>
    <td class="tg-42x8">23 Sep 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">Singapore</td>
    <td class="tg-42x8">0.993</td>
    <td class="tg-42x8">13 Sep 2021</td>
  </tr>
  <tr>
    <td class="tg-nzir">Finland</td>
    <td class="tg-nzir">0.999</td>
    <td class="tg-nzir">25 Dec 2021</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">South Korea</td>
    <td class="tg-nzir">0.998</td>
    <td class="tg-nzir">11 Nov 2021</td>
  </tr>
  <tr>
    <td class="tg-42x8">France</td>
    <td class="tg-42x8">0.997</td>
    <td class="tg-42x8">24 Dec 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">Spain</td>
    <td class="tg-42x8">0.998</td>
    <td class="tg-42x8">15 Sep 2021</td>
  </tr>
  <tr>
    <td class="tg-nzir">Guernsey</td>
    <td class="tg-nzir">0.994</td>
    <td class="tg-nzir">25 Dec 2021</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Taiwan</td>
    <td class="tg-nzir">0.999</td>
    <td class="tg-nzir">08 Feb 2022</td>
  </tr>
  <tr>
    <td class="tg-42x8">Iceland</td>
    <td class="tg-42x8">0.995</td>
    <td class="tg-42x8">06 Sep 2021</td>
    <td class="tg-i81m"></td>
    <td class="tg-42x8">United Arab Emirates</td>
    <td class="tg-42x8">0.995</td>
    <td class="tg-42x8">19 Aug 2021</td>
  </tr>
  <tr>
    <td class="tg-nzir">Ireland</td>
    <td class="tg-nzir">0.999</td>
    <td class="tg-nzir">01 Oct 2021</td>
    <td class="tg-quxf"></td>
    <td class="tg-nzir">Vietnam</td>
    <td class="tg-nzir">1.000</td>
    <td class="tg-nzir">21 Jan 2022</td>
  </tr>
</tbody>
</table></center>

In [ ]:
df1 = data_table[pd.to_datetime(data_table.Date).dt.strftime("%d") == '15' ]
df2 = data_table[pd.to_datetime(data_table.Date).dt.strftime("%d") == '01' ]
df = pd.concat([df1, df2], axis=0)
df.sort_values(by=['Date'], axis=0, ascending=True, inplace=True)

fig = px.choropleth(
                    df, 
                    locations="Country",
                    locationmode="country names",
                    color="Fully_Vaccinated_Ratio",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.thermal_r,
                    hover_data=["Fully_Vaccinated_Ratio"],
                    animation_frame='Date',
                    )

fig.update_layout(
                  height=700, 
                  coloraxis_colorbar=dict( title="Fully vaccinated, %"),
                  margin=dict(l=20,r=0,b=0,t=50,pad=0),
                  title={'text': 'Number of daily COVID-19 vaccinations worldwide', 'y':0.96, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  font_family="'Nunito', sans-serif",
                  )
fig.show()

<br> 
<h2 id="Results">Results</h2>  

--- 

<div align="justify">In this project, the countries that reached the threshold of herd immunity were found. For those countries that can reach this threshold, the date of the threshold attainment was calculated.    

<ul>
  <li>The vaccination rates were examined, cleaned and plotted for each country and continent. All the graphs in this project are interactive and can be thoroughly examined.</li>
  <li>The vaccination rates were fitted by logistic function. This regression fitted the data from 199 countries with \(R^2 > 0.9\). Other counties were not analyzed due to lack of data.</li>
  <li>Out of these 199 countries only 38 showed accomplishment of reaching the status of 75% immunized population, which is the lowest threshold for herd immunity for alpha variant of COVID-19.</li>
  <li>The mathematical model used to fit the data of vaccination rates shows that other countries cannot surpass the threshold of herd immunity.</li>
</ul>  </div>
  

<br><br><br>
<center><a href="https://konstantinburkin.github.io">Main page</a></center><br><br>